In [1]:
from tasks.linkage import Linkage

In [2]:
l = Linkage(10,0.2)
l.seed()
l.reset()

In [7]:
for link in l.linkages:
    print(len(link))

20
36
192
672
1613
2688
3072
2304
1024
205


In [19]:
l.step('1011111111')


99.64451581340256

In [5]:
l.linkages

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [20]:
import gomea
import numpy as np
# Custom fitness function resembling the concatenated deceptive trap function of size k
class CustomTrapFunction(gomea.fitness.BBOFitnessFunctionDiscrete):
    # Any members must be assigned in __new__ to make them accessible during instantiation of superclass
    def __new__(self, number_of_variables, k,l):
        self.k = k
        self.l = l
        return super().__new__(self,number_of_variables)
        
    def objective_function(self, objective_index, variables):
        f = 0
        s = ''
        for i in range(0,self.number_of_variables):
            s += str(variables[i])
        f = self.l.step(s)
        #print(f)
        return f

dim = 6
l = Linkage(dim,1)
l.seed(1)
l.reset()
lm = gomea.linkage.LinkageTree()
fd = CustomTrapFunction(dim,k=1,l=l)
dgom = gomea.DiscreteGOMEA(fitness=fd,linkage_model=lm,max_number_of_evaluations=1000)
result = dgom.run()
result.printAllStatistics()
result.printFinalStatistics()

SystemError: <class 'gomea.output.OutputStatistics'> returned a result with an exception set